# Example of a Router Chain in LangChain

https://python.langchain.com/docs/how_to/sequence/

Again, this example is based off the great course The Complete LangChain LLMs Guide, but I've updated the code to reflect the newer LangChain runnables format, and also to show which template was used in the answer

In [9]:
"""
Modern LangChain version using local Ollama model (gemma3:1b) and Runnable interfaces.

This script utilizes LangChain to create a multi-prompt chain framework. It leverages 
the Ollama local model for NLP tasks, enabling users to pose questions across four domains: 
biology, mathematics, astronomy, and travel. A simple routing mechanism identifies the 
most relevant expert model to answer the query, ensuring domain-specific expertise is applied.

Modules Used:
- langchain_ollama: Interface to a local Ollama LLM model.
- langchain_core.prompts: Provides tools to design structured, reusable prompt templates.
- langchain_core.runnables: Builds executable chains of operations using LLMs and auxiliary functions.
- langchain_core.output_parsers: Parses raw LLM output into user-friendly results.
"""

from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Step 1: Initialize the local Ollama LLM
# OllamaLLM provides access to the local model "gemma3:1b" to answer user questions.
# The `temperature=0.0` ensures deterministic responses (the same input yields the same output every time).
llm = OllamaLLM(model="gemma3:1b", temperature=0.0)

# Step 2: Define expert-specific prompt templates
# Each expert prompt specifies the conversational tone, domain of expertise, and fallback behavior.
# These templates guide the LLM in answering questions appropriately for each domain.

biology_template = """You are a very smart biology professor. 
You are great at answering questions about biology in a concise and easy-to-understand manner. 
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

astronomy_template = """You are a very good astronomer. You are great at answering astronomy questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

travel_agent_template = """You are a very good travel agent with a vast amount 
of knowledge when it comes to getting people the best deals and recommendations 
for travel, vacations, flights, and the world's best destinations for vacations. 
You are great at answering travel, vacation, flights, transportation, and tourist guides questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

# Step 3: Define metadata for expert models
# A list is created to store the names, descriptions, and prompt templates for each domain.
# This metadata will later be used to dynamically create chains for each expert.

prompt_infos = [
    {
        "name": "Biology",
        "description": "Good for answering biology-related questions",
        "prompt_template": biology_template
    },
    {
        "name": "Math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "Astronomy",
        "description": "Good for answering astronomy questions",
        "prompt_template": astronomy_template,
    },
    {
        "name": "Travel_Agent",
        "description": "Good for answering travel, tourism, and vacation questions",
        "prompt_template": travel_agent_template,
    },
]

# Step 4: Build domain-specific chains
# This step creates a reusable chain for each expert, combining the prompt template, 
# LLM, and an output parser to structure responses. A dictionary maps each chain to its domain.

destination_chains = {}
for info in prompt_infos:
    name = info["name"]
    template = ChatPromptTemplate.from_template(info["prompt_template"])
    chain = template | llm | StrOutputParser()  # Combines prompt, LLM, and parser into a single chain
    destination_chains[name.lower()] = chain  # Normalize names to lowercase for consistency in routing

# Step 5: Define a default chain for unmatched queries
# When a query doesn't match any domain-specific keywords, this fallback chain handles it.
# It uses a generic prompt template to structure the response.
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = default_prompt | llm | StrOutputParser()

# Step 6: Implement a router function
# The router analyzes the input query for keywords and assigns it to the most relevant domain.
# If no keywords match, the router assigns the query to the default chain.
def simple_router(input: str) -> dict:
    """
    Routes the input to the appropriate expert domain based on keywords in the query.
    
    Args:
        input (str): User's query as a string.
    
    Returns:
        dict: A dictionary containing the determined destination domain and the original query.
    """
    input_lower = input.lower()
    if "biology" in input_lower or "cell" in input_lower or "gene" in input_lower:
        destination = "biology"
    elif "math" in input_lower or "calculate" in input_lower or "equation" in input_lower:
        destination = "math"
    elif "star" in input_lower or "planet" in input_lower or "astronomy" in input_lower:
        destination = "astronomy"
    elif "vacation" in input_lower or "travel" in input_lower or "flight" in input_lower:
        destination = "travel_agent"
    else:
        destination = None  # Defaults to the fallback chain
    return {"destination": destination, "input": input}

# Step 7: Combine routing with chain execution
# RunnableLambda applies the router function, and RunnableMap integrates chain execution logic.
# The input query is dynamically routed to the appropriate expert chain based on keywords.
router = RunnableLambda(simple_router)

multi_prompt_chain = (
    router |
    RunnableMap({
        # Structure the result: include the destination domain and LLM output.
        "destination": lambda x: x["destination"] or "default",  # Assign "default" if no domain matches
        "output": lambda x: destination_chains.get(x["destination"], default_chain).invoke(x["input"])
    })
)

# Step 8: Test the multi-prompt chain with a sample query
if __name__ == "__main__":
    user_question = "What do white blood cells do?"
    result = multi_prompt_chain.invoke(user_question)
    
    # Display the determined expert domain and the generated response.
    print(f"\n🔎 Selected Expert: {result['destination'].capitalize()}")
    print("🧠 Answer:")
    print(result["output"])


🔎 Selected Expert: Biology
🧠 Answer:
Alright, that’s a fantastic question! White blood cells, or leukocytes, are incredibly important in your body. Essentially, they’re your body’s defense force. Here’s a breakdown of what they do:

**Their main roles are:**

*   **Immunity:** They’re constantly patrolling your body, looking for signs of infection or disease.
*   **Fighting Infections:** They release chemicals like histamine to fight bacteria and viruses.
*   **Removing Waste:** They engulf and destroy cellular debris and damaged cells.
*   **Repairing Tissue:** Some types help with wound healing.

**In short, they’re your body’s first line of defense against anything that doesn’t belong!**

Do you have any more specific questions about white blood cells, perhaps about their different types or how they function?
